In [ ]:
import pandas as pd
import numpy as np
import sklearn
import os
import glob
from datetime import datetime, timedelta

In [ ]:
def is_reachable(T, S, c):
    return c['trip_id'] in T or (c['departure_stop'] in S and S[c['departure_stop']] < c['departure_time'])

def connection_scan(source, target, start_time, footpaths, connections):
    S = {}
    T = {}
    J = {}
    S[source] = start_time
    for i, f in footpaths[footpaths['from_stop_id'] == source].iterrows():
        S[i] = start_time + timedelta(minutes=int(f.transfer_time))
    
    for _, c in connections.iterrows():
        if c.arrival_stop == '8590723' and c.departure_stop == '8557033':
            print(c)
        if target in S and S[target] <= c['departure_time']:
            break
        
        if is_reachable(T, S, c):
            if c['trip_id'] not in T:
                T[c['trip_id']] = c
            for i, f in footpaths[footpaths['from_stop_id'] == c['arrival_stop']].iterrows():
                transfer_time = f.transfer_time
                if i not in S or c['arrival_time'] + timedelta(minutes=int(transfer_time)) < S[i]:
                    S[i] = c['arrival_time'] + timedelta(minutes=int(transfer_time))
                    J[i] = (T[c['trip_id']], c, transfer_time)
    return J

def get_journey(source, target, J, footpaths):
    journey = []
    t = target
    while t in J and J[t] is not None:
        journey = [J[t]] + journey
        t = J[t][0]['departure_stop']
    # if footpaths[source][target] != 0:
    #     journey = [footpaths[source][target]] + journey
    return journey

In [ ]:
connections_df = pd.read_csv('../data/connections.csv')
transfers_df = pd.read_csv('../data/transfers.csv')

In [ ]:
today = '2023-05-21'
now = datetime(year=2023, month=5, day=21, hour=12, minute=0)

transform_today_time = lambda col: lambda row: datetime.fromisoformat(f'{today} {row[col]}')

connections_df['departure_time'] = connections_df.apply(transform_today_time('departure_time'), axis=1)
connections_df['arrival_time'] = connections_df.apply(transform_today_time('arrival_time'), axis=1)

In [ ]:
J = connection_scan('8557033', '8573711', datetime(year=2023, month=5, day=21, hour=6, minute=0), transfers_df, connections_df.sort_values(by='departure_time'))